In [169]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import talib

sns.set_style('darkgrid')

In [170]:
nvda = yf.Ticker("NVDA")
nvda_data = nvda.history(period="5y")

df = nvda_data[["Close"]]
df.index = pd.to_datetime(df.index.date)

lookback = 20

df['SMA'] = talib.SMA(df['Close'], timeperiod=lookback)

/var/folders/lg/qshzdl1s03z_hdz077pw4nbm0000gn/T/ipykernel_24108/278006168.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA'] = talib.SMA(df['Close'], timeperiod=lookback)


In [171]:
from sklearn.preprocessing import StandardScaler, scale

for i in range(1, lookback + 1):
    df[f"c{i}"] = df['Close'].shift(i)

scaler = StandardScaler()

df[df.columns] = scaler.fit_transform(df[df.columns])

df.dropna(inplace=True)

/var/folders/lg/qshzdl1s03z_hdz077pw4nbm0000gn/T/ipykernel_24108/1192117267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"c{i}"] = df['Close'].shift(i)
/var/folders/lg/qshzdl1s03z_hdz077pw4nbm0000gn/T/ipykernel_24108/1192117267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"c{i}"] = df['Close'].shift(i)
/var/folders/lg/qshzdl1s03z_hdz077pw4nbm0000gn/T/ipykernel_24108/1192117267.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .l

In [172]:
df['target'] = df['Close']
df.drop(['Close','SMA'], axis=1, inplace=True)

train_df = df.iloc[:int(df.shape[0] * 0.9)]
test_df = df.iloc[int(df.shape[0] * 0.9):]

In [173]:
train_df.shape, test_df.shape

((1115, 21), (124, 21))

In [174]:
import torch 

X_train = torch.from_numpy(train_df.iloc[:, :-1].values).float().reshape(-1, 1, train_df.shape[1] - 1)
y_train = torch.from_numpy(train_df.iloc[:, -1].values).float()
X_test = torch.from_numpy(test_df.iloc[:, :-1].values).float().reshape(-1, 1, test_df.shape[1] - 1)
y_test = torch.from_numpy(test_df.iloc[:, -1].values).float()

X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([1115, 1, 20]),
 torch.Size([1115]),
 torch.Size([124, 1, 20]),
 torch.Size([124]))

In [175]:
from torch.utils.data import TensorDataset, DataLoader

train_ds = TensorDataset(X_train, y_train)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=False)
test = TensorDataset(X_test, y_test)
test_dl = DataLoader(test, batch_size=64, shuffle=False)

In [176]:
class GRU(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.gru = torch.nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, hn = self.gru(x, h0.detach())
        out = self.fc(out[:, -1, :])
        return out

In [177]:
model = GRU(input_dim=lookback, hidden_dim=64, layer_dim=2, output_dim=1)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [178]:
max_epochs = 100

for epoch in range(max_epochs):
    for i, (inputs, labels) in enumerate(train_dl):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item():.4f}')

/opt/homebrew/Caskroom/miniconda/base/envs/dl/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniconda/base/envs/dl/lib/python3.10/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([27])) that is different to the input size (torch.Size([27, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Loss: 0.5514
Epoch: 2, Loss: 0.5933
Epoch: 3, Loss: 1.4572
Epoch: 4, Loss: 0.1777
Epoch: 5, Loss: 0.1782
Epoch: 6, Loss: 0.1752
Epoch: 7, Loss: 0.1746
Epoch: 8, Loss: 0.1738
Epoch: 9, Loss: 1.8555
Epoch: 10, Loss: 0.2123
Epoch: 11, Loss: 0.1737
Epoch: 12, Loss: 0.1754
Epoch: 13, Loss: 0.1732
Epoch: 14, Loss: 0.2298
Epoch: 15, Loss: 0.1804
Epoch: 16, Loss: 0.2211
Epoch: 17, Loss: 0.2716
Epoch: 18, Loss: 0.1904
Epoch: 19, Loss: 0.2212
Epoch: 20, Loss: 0.1995
Epoch: 21, Loss: 0.2507
Epoch: 22, Loss: 0.2301
Epoch: 23, Loss: 0.1818
Epoch: 24, Loss: 0.1735
Epoch: 25, Loss: 0.1904
Epoch: 26, Loss: 0.2213
Epoch: 27, Loss: 0.1780
Epoch: 28, Loss: 0.1825
Epoch: 29, Loss: 0.1741
Epoch: 30, Loss: 0.1788
Epoch: 31, Loss: 0.2198
Epoch: 32, Loss: 0.1765
Epoch: 33, Loss: 0.1827
Epoch: 34, Loss: 0.1737
Epoch: 35, Loss: 0.1817
Epoch: 36, Loss: 0.2243
Epoch: 37, Loss: 0.1770
Epoch: 38, Loss: 0.1786
Epoch: 39, Loss: 0.1741
Epoch: 40, Loss: 0.1861
Epoch: 41, Loss: 0.2237
Epoch: 42, Loss: 0.1757
E

In [179]:
model = model.eval()
y_pred = model(X_test)

from sklearn.metrics import mean_squared_error

np.sqrt(mean_squared_error(y_test.detach().numpy(), y_pred.detach().numpy()))

0.6004334

In [203]:
y = y_pred.detach().numpy()
z = (y * scaler.scale_[-1] + scaler.mean_[-1])

output = z.reshape(-1)[-10:]

final_df = pd.DataFrame(output, columns=['Close'])
final_df.to_csv('output.csv')